In [ ]:
import pandas as pd
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

import gc

df = pd.read_csv("../input/ubiquant-market-prediction/train.csv", nrows=99999)
df.head(5)

#gc.collect(generation= 2)


In [ ]:
#check the missing values
temp_mv = pd.DataFrame(df.isna().sum())
from collections import Counter
Counter(temp_mv[0],axis=1)

#thre is no missing values


In [ ]:
#remove the row_id and time_id
df.drop(["row_id","time_id","investment_id"],axis=1,inplace=True)

#df.head(10)
print(df.shape)


In [ ]:
gc.collect(generation=2)

#define X and y
X = df.drop("target", axis=1)
y = df["target"]

print(X.columns)
y.head(3)

In [ ]:
#correct Sknewness
import numpy as np
from scipy.stats import skew

#sample
print("Skewness before",skew(X["f_0"]))

for i in X.columns:
    s_v = skew(X[i])
    if (s_v >=0.2 or s_v <= 0.2):
        #print("yes")
        X[i] = np.sqrt(X[i])
    else:
        pass
    
#sample    
print("Skewness After", skew(X["f_0"]))

In [ ]:

#check the skewness
for i in X.columns:
    print(i)
    sns.displot(X[i])
    
# the reason i m not using corrected skewed data is becoz need to remove few columns which have 
#way to imbalanced distribution using Nan, -inf , large number to float error during Model training.


In [ ]:
#understanding the format of the data
df.dtypes

In [ ]:
#corr relation check
corr_matrix = X.corr().abs()
sns.heatmap(corr_matrix)


In [ ]:
import numpy as np
print("Shape of the dataset before removing colineartiy", X.shape)
#select upper traingle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape),k=1).astype(np.bool))
#find features with correlation greater than 0.80
to_drop = [column for column in upper.columns if any(upper[column]>0.80)]
#Drop the features
X.drop(to_drop,axis=1,inplace=True)

print("Shape of the dataset after removing colineartiy", X.shape)

In [ ]:
#train and test data
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25, random_state=1)

In [ ]:
#scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf_model = RandomForestRegressor()
rf_model.fit(X_train,y_train)

y_pred = rf_model.predict(X_test)

from sklearn import metrics
print("Mean Squared Error", metrics.mean_squared_error(y_test,y_pred))
print("Roo)t Mean Squared Error", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
print("Mean Aboslute Score", metrics.mean_absolute_error(y_test,y_pred)

In [ ]:
#Grid Search 
from sklearn.model_selection import RandomizedSearchCV

parameters = [{'n_estimators':[100,300,500,700,1000,1500],'max_depth':[10, 100, 500, 1000],'min_samples_split':[2,3,4,5,6,7,8,9,10],'min_samples_leaf':[2,3,4,5],'min_impurity_decrease':[2,3,4,5],
              'max_features':["auto", "sqrt", "log2"]}]
grid_search = RandomizedSearchCV(estimator = rf_model, param_distributions = parameters, cv = 3) 
grid_search.fit(X_train,y_train)
print("Best Accuracy ", grid_search.best_score_)
best_parameters = grid_search.best_params_


In [ ]:

rf_model = RandomForestRegressor(n_estimators =best_parameters["n_estimators"],min_samples_split=best_parameters["min_samples_split"], max_features=best_parameters["max_features"],
                                  min_samples_leaf=best_parameters["min_samples_leaf"],max_depth =best_parameters["max_depth"],min_impurity_decrease=best_parameters["min_impurity_decrease"], random_state = 0)
rf_model.fit(X_train,y_train)
y_pred = rf_model.predict(X_test)

In [ ]:
#we can visualize the predicted results with the actual results
df_results = pd.DataFrame({'Actual':y_test.ravel(),'Predicted':y_pred.ravel()})
print(df_results)

In [ ]:
import matplotlib.pyplot as plt
#we can also visualize the actual vs predicted
df_1 = df_results.head(25)
df_1.plot(kind='bar',figsize=(16,10))
plt.grid(which='major', linestyle='-', linewidth='0.5', color='green')
plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
plt.show()

In [ ]:
from sklearn import metrics
print("Mean Squared Error", metrics.mean_squared_error(y_test,y_pred))
print("Roo)t Mean Squared Error", np.sqrt(metrics.mean_squared_error(y_test,y_pred)))
print("Mean Aboslute Score", metrics.mean_absolute_error(y_test,y_pred)